In [1]:
from datetime import datetime
import itertools
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras_preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Embedding, Conv1D, GlobalMaxPooling1D, Flatten, LSTM, \
    Bidirectional, MaxPooling1D
from keras.preprocessing import text, sequence
from keras import utils
import pandas as pd

from utility.train_data_loader import load_train_data


testData = pd.read_csv("../data/test.csv")
dictData = pd.read_csv("../data/kata_dasar_kbbi.csv")
categories_file = open("../data/categories.json", "r")
categories = json.load(categories_file)
inverted_categories_mobile = {v: k.lower() for k, v in categories['Mobile'].items()}
inverted_categories_fashion = {v: k.lower() for k, v in categories['Fashion'].items()}
inverted_categories_beauty = {v: k.lower() for k, v in categories['Beauty'].items()}

all_subcategories = {k.lower(): v for k, v in categories['Mobile'].items()}
all_subcategories.update({k.lower(): v for k, v in categories['Fashion'].items()})
all_subcategories.update({k.lower(): v for k, v in categories['Beauty'].items()})

# Main settings
plot_history_check = True
gen_test = True
max_length = 35  # 32 is max word in train
max_words = 2500
num_classes = len(all_subcategories)
# Training for more epochs will likelval-acc after 10 epochs: 0.71306y lead to overfitting on this dataset
# You can try tweaking these hyperparamaters when using this model with your own data
batch_size = 256
epochs = 10

print(all_subcategories)
print("no of categories: " + str(num_classes))

category_mapping = {
    'fashion_image': 'Fashion',
    'beauty_image': 'Beauty',
    'mobile_image': 'Mobile',
}
directory_mapping = {
    'Fashion': 'fashion_image',
    'Beauty': 'beauty_image',
    'Mobile': 'mobile_image',
}

trainData = load_train_data()

Using TensorFlow backend.


{'others mobile & tablet': 35, 'smartfren': 53, 'infinix': 40, 'brandcode': 39, 'icherry': 52, 'advan': 45, 'iphone': 31, 'realme': 51, 'motorola': 49, 'maxtron': 56, 'nokia': 38, 'xiaomi': 34, 'mito': 46, 'sony': 33, 'spc': 57, 'lenovo': 37, 'alcatel': 55, 'samsung': 32, 'vivo': 42, 'evercoss': 44, 'strawberry': 50, 'blackberry': 36, 'asus': 43, 'honor': 54, 'oppo': 41, 'huawei': 47, 'sharp': 48, 'wedding dress': 23, 'shirt': 27, 'casual dress': 18, 'maxi dress': 20, 'big size dress': 24, 'bodycon dress': 22, 'party dress': 19, 'blouse': 26, 'tshirt': 25, 'crop top': 29, 'tanktop': 28, 'others': 17, 'a line dress': 21, 'big size top': 30, 'foundation': 1, 'face palette': 0, 'concealer': 7, 'lip gloss': 14, 'blush on': 2, 'highlighter': 8, 'bb & cc cream': 5, 'other face cosmetics': 4, 'lip tint': 13, 'bronzer': 11, 'lip liner': 15, 'powder': 3, 'setting spray': 10, 'primer': 9, 'contour': 6, 'other lip cosmetics': 16, 'lipstick': 12}
no of categories: 58
custom train data used


In [2]:
trainData.head()


,itemid,title,Category,image_path,item_category
0,307504,nyx sex bomb pallete natural palette,0,beauty_image/6b2e9cbb279ac95703348368aa65da09.jpg,face palette
1,461203,etude house precious mineral any cushion pearl...,1,beauty_image/20450222d857c9571ba8fa23bdedc8c9.jpg,foundation
2,3592295,milani rose powder blush,2,beauty_image/6a5962bed605a3dd6604ca3a4278a4f9.jpg,blush on
3,4460167,etude house baby sweet sugar powder,3,beauty_image/56987ae186e8a8e71fcc5a261ca485da.jpg,powder
4,5853995,bedak revlon color stay aqua mineral make up,3,beauty_image/9c6968066ebab57588c2f757a240d8b9.jpg,powder


In [3]:
# Shuffle train data
trainData = shuffle(trainData)

max_data_size = int(len(trainData) * 1)
train_data_size = int(max_data_size * .9)
train_data_step = 1
validate_data_step = 1
print(train_data_size, max_data_size)

train_texts = trainData['title'][:train_data_size:train_data_step]
valid_texts = trainData['title'][train_data_size::train_data_step]
train_tags = trainData['Category'][:train_data_size:train_data_step]
valid_tags = trainData['Category'][train_data_size::train_data_step]
test_texts = testData['title']
print(len(train_texts), len(train_tags))

y = train_tags.values

tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_texts)  # only fit on train
x_train = tokenize.texts_to_sequences(train_texts)
x_valid = tokenize.texts_to_sequences(valid_texts)
x_test = tokenize.texts_to_sequences(test_texts)

word_index = tokenize.word_index

# Pad sequences with zeros
x_train = pad_sequences(x_train, padding='post', maxlen=max_length)
x_valid = pad_sequences(x_valid,padding='post',maxlen=max_length)
x_test = pad_sequences(x_test, padding='post', maxlen=max_length)

y_train = train_tags.values
y_valid = valid_tags.values
vocab_size = len(tokenize.word_index) + 1
print(vocab_size)


599953 666615
599953 599953
41597


In [3]:
import xgboost

In [ ]:
accuracy = xgboost.XGBClassifier(verbosity=3).fit(train_texts,y_train,x_valid)

In [4]:
x_train.shape

(599953, 35)

In [5]:
y_train.shape

(599953,)